In [1]:
import tensorflow as tf

----------

In [2]:
filt_complex = tf.constant([2.5, 1, 3, 0.5, 1, 3, 0.5, 1, 3, 0.5, 1, 3, 0.5, 1, 3, 0.5, 1, 3, 0.5, 1, 3, 0.5, 1, 3, 0.5, 1, 3])
filt_complex

<tf.Tensor: shape=(27,), dtype=float32, numpy=
array([2.5, 1. , 3. , 0.5, 1. , 3. , 0.5, 1. , 3. , 0.5, 1. , 3. , 0.5,
       1. , 3. , 0.5, 1. , 3. , 0.5, 1. , 3. , 0.5, 1. , 3. , 0.5, 1. ,
       3. ], dtype=float32)>

In [3]:
Fiabs = tf.abs(filt_complex)
Fiabs

<tf.Tensor: shape=(27,), dtype=float32, numpy=
array([2.5, 1. , 3. , 0.5, 1. , 3. , 0.5, 1. , 3. , 0.5, 1. , 3. , 0.5,
       1. , 3. , 0.5, 1. , 3. , 0.5, 1. , 3. , 0.5, 1. , 3. , 0.5, 1. ,
       3. ], dtype=float32)>

In [4]:
filtersize=3
k = 0

Fiabs[k::filtersize]

<tf.Tensor: shape=(9,), dtype=float32, numpy=array([2.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], dtype=float32)>

In [37]:
# one graph at a time
filt_complex = tf.constant([2.5, 1, 3, 0.5, 1, 3, 0.5, 1, 3, 0.5, 1, 3, 0.5, 1, 3, 0.5, 1, 3, 0.5, 1, 3, 0.5, 1, 3, 0.5, 1, 3])
Fiabs = tf.abs(filt_complex)
filtersize=3
k = 0

Fiabs[k::filtersize]

threshold = 1.1
deviance = 0
for k in range(filtersize):
    timestep = Fiabs[k::filtersize]
    above_thres = tf.greater(timestep, threshold)
    below_thres = tf.less(timestep, threshold)
    prior_connection = tf.reduce_any(above_thres) # reduce_any returns if there is a True in the Tensor
    # If prior_connection == False
    # Calculate the sum of the weights that are below the threshold in the next timestep
    if prior_connection == False:
        deviance += tf.reduce_sum(tf.boolean_mask(Fiabs[k+1::filtersize], below_thres))
    print(timestep)
    print(above_thres)
    print(prior_connection)
    print("deviance:", deviance)
    print("\n")

tf.Tensor([2.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5], shape=(9,), dtype=float32)
tf.Tensor([ True False False False False False False False False], shape=(9,), dtype=bool)
tf.Tensor(True, shape=(), dtype=bool)
deviance: 0


tf.Tensor([1. 1. 1. 1. 1. 1. 1. 1. 1.], shape=(9,), dtype=float32)
tf.Tensor([False False False False False False False False False], shape=(9,), dtype=bool)
tf.Tensor(False, shape=(), dtype=bool)
deviance: tf.Tensor(27.0, shape=(), dtype=float32)


tf.Tensor([3. 3. 3. 3. 3. 3. 3. 3. 3.], shape=(9,), dtype=float32)
tf.Tensor([ True  True  True  True  True  True  True  True  True], shape=(9,), dtype=bool)
tf.Tensor(True, shape=(), dtype=bool)
deviance: tf.Tensor(27.0, shape=(), dtype=float32)




## While loop to parallelise graphs


In [21]:
filt_complex = tf.constant([2.5, 1, 3, 0.5, 1, 3, 0.5, 1, 3, 0.5, 1, 3, 0.5, 1, 3, 0.5, 1, 3, 0.5, 1, 3, 0.5, 1, 3, 0.5, 1, 3])
Fiabs = tf.abs(filt_complex)
filtersize = 3
threshold = 1.1
deviances = []
k=0
while k < filtersize:
    timestep = Fiabs[k::filtersize]
    print(timestep)
    above_thres = tf.cast(tf.greater(timestep, threshold), tf.bool)
    below_thres = tf.cast(tf.less(timestep, threshold), tf.bool)
    print(above_thres)
    prior_connection = tf.reduce_any(above_thres)
    
    if not prior_connection:
        deviances.append(tf.reduce_sum(tf.boolean_mask(Fiabs[k+1::filtersize], below_thres)))
    
    k += 1

deviance = tf.reduce_sum(deviances)
deviance.numpy()


tf.Tensor([2.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5], shape=(9,), dtype=float32)
tf.Tensor([ True False False False False False False False False], shape=(9,), dtype=bool)
tf.Tensor([1. 1. 1. 1. 1. 1. 1. 1. 1.], shape=(9,), dtype=float32)
tf.Tensor([False False False False False False False False False], shape=(9,), dtype=bool)
tf.Tensor([3. 3. 3. 3. 3. 3. 3. 3. 3.], shape=(9,), dtype=float32)
tf.Tensor([ True  True  True  True  True  True  True  True  True], shape=(9,), dtype=bool)


27.0

In [20]:
def filter_deviance(filter, filtersize):
    """Calculates the deviance of a sparse graph where deviance is
    defined as when a graph has no feeder event or following
    connections.

    Args:
        filter (tf.Tensor): Weights from the graph.
        filter_size (int): Number of time steps (t) to use in 3D CNN filters.

    Returns:
        float: Unscaled deviance from the so-called 'perfect' graph.
    """
    Fiabs = tf.abs(filter)
    threshold = 1.1
    deviances = []
    k=0
    while k < filtersize:
        timestep = Fiabs[k::filtersize]
        above_thres = tf.cast(tf.greater(timestep, threshold), tf.bool)
        below_thres = tf.cast(tf.less(timestep, threshold), tf.bool)
        prior_connection = tf.reduce_any(above_thres)
        
        if not prior_connection:
            deviances.append(tf.reduce_sum(tf.boolean_mask(Fiabs[k+1::filtersize], below_thres)))
        
        k += 1

    deviance = tf.reduce_sum(deviances)
    
    return deviance.numpy()

filter_deviance(filt_complex, filtersize=3)

27.0